## Estatísticas Anuais - CID-10 de auxílios por incapacidade temporária "previdenciários" (sem relação com o trabalho) 

Arquivos (Acidentários) baixados de: https://www.gov.br/trabalho-e-previdencia/pt-br/assuntos/previdencia-social/saude-e-seguranca-do-trabalhador/dados-de-acidentes-do-trabalho/tabelas-cid-10

### Link dos datasets utilizados e gerados por este notebook

https://drive.google.com/drive/folders/1ZvH_MjFpXiVSVAV-RT_TxpaY5T9pFnWu?usp=sharing

Este notebook realiza algumas transformações e limpezas dos datasets obtidos do Ministério do Trabalho e Previdência.

Ao fim dos processos, pode-se obter subconjuntos de dados mais limpos, podendo fazer a junção por colunas com totais anuais, filtrar as categorias de interesse, bem como gerar subsets por seleção de termos específicos em determinadas linhas.

Devido ao modelo dos datasets disponibilizados pelo referido órgão, os processos abaixo só se aplicam aos dados de 2017 em diante. Dados anteriores a 2017 estão disponíveis no site acima em formato pdf.

In [1]:
# Imports
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Funções

# Função ETL

def etl_data(df):
    '''Executa algumas transformações no dataset original e o salva como csv'''    
        
    # Exclui até linha de índice 12 e altera linha de índice 13 como cabeçalho.
    df.drop(df.index[0:13], inplace=True)
    df = df.rename(columns = df.iloc[0]).drop(df.index[0])    
    
    # Exclui as últimas 5 linhas
    df.drop(df.index[2070:], inplace=True)    
    
    # Converte para float64 e caracteres '-' são considerandos NaN
    colunas = ['Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 'Julho','Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']
    df = df.reindex(columns=['CID10 CATEGORIA','Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 'Julho','Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro'])
    df[colunas] = df[colunas].apply(pd.to_numeric, errors='coerce')    

    # Substitui Nan por 0
    df.fillna(0, inplace = True)    
    
    # Cria a coluna total do ano
    df['Total_' + str(i)] = df.sum(axis=1)    
    
    # set_index. Coluna 'CID10 CATEGORIA' como index
    #df = df.set_index('CID10 CATEGORIA')       
    
    # Salva o dataframe com as alterações no diretório /datasets_tratados
    #df.to_csv('datasets_tratados/' + str(i) + '.csv', sep=';', encoding='utf-8')      
    
    # Sugestão para salvar apenas com os totais anuais
    df.to_csv('datasets_tratados/' + str(i) + '.csv', sep=';', encoding='utf-8', columns = ['CID10 CATEGORIA', 'Total_' + str(i)], index=False) 
    

###########################################################################################################################
###########################################################################################################################

# Use para selecionar categorias desejadas
def selecionaCategoria(character):
    '''Cria uma tupla com os códigos da categoria. Em character, digitar somente letras de A-Z (entre aspas)'''
    character = character.upper()
    list = []
    for i in range(100):
        if i < 10:
            list.append(str(character) + str('0') + str(i))
        else:
            list.append(str(character) + str(i))
    my_tuple = tuple(list)
    return my_tuple


In [3]:
# Carrega os datasets originais e aplica a função etl_data()
for i in range(2017, 2022):
    path = 'datasets_originais/' +str(i) + '-cid10.xlsx'
    df = pd.read_excel(path)
    etl_data(df)

In [4]:
# Substitui os caracteres carregados errados devido à codificação
for i in range(2017, 2022):
    v = 'datasets_tratados/' + str(i) + '.csv'
    df = pd.read_csv(v, sep=';', on_bad_lines='skip')
    df['CID10 CATEGORIA'] = df['CID10 CATEGORIA'].map(lambda x: x.replace('?E', 'oe'))
    df['CID10 CATEGORIA'] = df['CID10 CATEGORIA'].map(lambda x: x.replace('?O', 'ao'))
    df['CID10 CATEGORIA'] = df['CID10 CATEGORIA'].map(lambda x: x.replace('ç', 'c'))
    df['CID10 CATEGORIA'] = df['CID10 CATEGORIA'].map(lambda x: x.replace('ã', 'a'))
    df['CID10 CATEGORIA'] = df['CID10 CATEGORIA'].map(lambda x: x.replace('õ', 'o')) 
    
        
    df.to_csv('datasets_tratados/' + str(i) + '.csv', sep=';', encoding='utf-8', columns = ['CID10 CATEGORIA', 'Total_' + str(i)], index=False)

In [5]:
# Confere o arquivo salvo
a = pd.read_csv('datasets_tratados/2021.csv', sep=';', on_bad_lines='skip')

In [6]:
a.head(10)

,CID10 CATEGORIA,Total_2021
0,Total,1830419.0
1,Capítulo I: Algumas doencas Infecciosas e para...,135433.0
2,A00:Colera,61.0
3,A01:Febres Tifoide e Paratifoide,4.0
4,A02:Outras Infeccoes por Salmonella,5.0
5,A04:Outras Infeccoess Intestinais Bacterianas,20.0
6,A05:Outras Intoxicacoes Alimentares Bacterianas,2.0
7,A06:Amebiase,19.0
8,A07:Outras Doencas Intestinais por Protozoarios,5.0
9,"A08:Infeccoes Intestinais Virais, Outras e as ...",10.0


In [7]:
a.tail(20)

,CID10 CATEGORIA,Total_2021
1695,Z86:Historia Pessoal de Algumas Outras Doencas,3.0
1696,Z87:Historia Pessoal de Outras Doencas e Afeccoes,14.0
1697,Z89:Ausencia Adquirida de Membros,44.0
1698,Z90:Ausencia Adquirida de Orgaos Nao Classific...,29.0
1699,"Z91:Historia Pessoal de Fatores de Risco, Nao ...",1.0
1700,Z93:Orificios Artificiais,265.0
1701,Z94:Orgaos e Tecidos Transplantados,544.0
1702,Z95:Presenca de Implantes e Enxertos Cardiacos...,183.0
1703,Z96:Presenca de Outros Implantes Funcionais,57.0
1704,Z97:Presenca de Outros Dispositivos Proteticos,10.0


## Leitura dos Dataframes e Armazenamento em Variáveis

In [8]:
# Cria uma lista com o path dos datasets tratados
lista_csv = []
for i in range(2017, 2022):
    u = 'datasets_tratados/' +str(i) + '.csv'    
    lista_csv.append(u)

In [9]:
lista_csv

['datasets_tratados/2017.csv',
 'datasets_tratados/2018.csv',
 'datasets_tratados/2019.csv',
 'datasets_tratados/2020.csv',
 'datasets_tratados/2021.csv']

In [10]:
# Cria dataframes para cada ano
df2017 = pd.read_csv(lista_csv[0], sep=';', on_bad_lines='skip')
df2018 = pd.read_csv(lista_csv[1], sep=';', on_bad_lines='skip')
df2019 = pd.read_csv(lista_csv[2], sep=';', on_bad_lines='skip')
df2020 = pd.read_csv(lista_csv[3], sep=';', on_bad_lines='skip')
df2021 = pd.read_csv(lista_csv[4], sep=';', on_bad_lines='skip')

In [11]:
print(df2017.shape)
print(df2018.shape)
print(df2019.shape)
print(df2020.shape)
print(df2021.shape)

(2070, 2)
(2070, 2)
(2070, 2)
(2070, 2)
(1715, 2)


## Seleciona Categorias

In [12]:
# Remove linhas finais e outras
df2017.drop(df2017.index[2067:], inplace = True)
df2018.drop(df2018.index[2067:], inplace = True)
df2019.drop(df2019.index[2067:], inplace = True)
df2020.drop(df2020.index[2067:], inplace = True)
df2021.drop(df2021.index[754:], inplace = True)

words = ['Total', 'Capítulo', 'Capitulo']
pattern2 = '|'.join(words)

df2017 = df2017[~df2017['CID10 CATEGORIA'].str.contains(pattern2)]
df2018 = df2018[~df2018['CID10 CATEGORIA'].str.contains(pattern2)]
df2019 = df2019[~df2019['CID10 CATEGORIA'].str.contains(pattern2)]
df2020 = df2020[~df2020['CID10 CATEGORIA'].str.contains(pattern2)]
df2021 = df2021[~df2021['CID10 CATEGORIA'].str.contains(pattern2)]

KeyError: "['Total_2017'] not in index"

In [ ]:
# Seleciona as categorias desejadas
catF = selecionaCategoria('f')
catZ = selecionaCategoria('z')

# Fornece prints para criar novos subsets com as referidas categorias
cats = ['F', 'Z']
for ano in range(2017, 2022):
    for cat in cats:
        print(f"df{ano}_{cat} = df{ano}[df{ano}['CID10 CATEGORIA'].str.startswith(cat{cat.upper()})]")

In [ ]:
# Copia e cola o resultado acima para gerar os novos subsets

df2017_F = df2017[df2017['CID10 CATEGORIA'].str.startswith(catF)]
df2017_Z = df2017[df2017['CID10 CATEGORIA'].str.startswith(catZ)]
df2018_F = df2018[df2018['CID10 CATEGORIA'].str.startswith(catF)]
df2018_Z = df2018[df2018['CID10 CATEGORIA'].str.startswith(catZ)]
df2019_F = df2019[df2019['CID10 CATEGORIA'].str.startswith(catF)]
df2019_Z = df2019[df2019['CID10 CATEGORIA'].str.startswith(catZ)]
df2020_F = df2020[df2020['CID10 CATEGORIA'].str.startswith(catF)]
df2020_Z = df2020[df2020['CID10 CATEGORIA'].str.startswith(catZ)]
df2021_F = df2021[df2021['CID10 CATEGORIA'].str.startswith(catF)]
df2021_Z = df2021[df2021['CID10 CATEGORIA'].str.startswith(catZ)]

In [ ]:
# Escolhendo-se mais de duas categorias, usa-se o método concat() para unir as categorias por ano.
d1 = pd.concat([df2017_F, df2017_Z])
d2 = pd.concat([df2018_F, df2018_Z])
d3 = pd.concat([df2019_F, df2019_Z])
d4 = pd.concat([df2020_F, df2020_Z])
d5 = pd.concat([df2021_F, df2021_Z])
d2

## Merge para Unir os Subsets Criados

In [ ]:
print(d1.shape)
print(d2.shape)
print(d3.shape)
print(d4.shape)
print(d5.shape)

In [ ]:
# Cuidado ao unir o d5 (ano de 2021), pois apresenta menos registros
from functools import reduce
data_frames = [d1, d2, d3, d4]

#df_merged = reduce(lambda  left,right: pd.merge(left,right, left_index=True, right_index=True, how='right'), data_frames)
    
df_merged = reduce(lambda  left,right: pd.merge(left,right, on=['CID10 CATEGORIA'],how='right'), data_frames)
df_merged

In [ ]:
df_merged.isna().sum()

In [ ]:
# Selecionando linhas específicas
termos = ['Neuroticos', 'ansiosos', 'maquinas']
pattern = '|'.join(termos)

df_merged[df_merged['CID10 CATEGORIA'].str.contains(pattern, case = False, na = False)]